In [1]:
import pandas as pd
import numpy as np

In [2]:
# read Land Parcel data (csv) in as a pandas dataframe
land_parcel_df = pd.read_csv('../data/land_parcel/mapc.ma_parcels_metrofuture.csv')

land_parcel_df.head()

C:\Users\xsmil\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6,14,17,18,29,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,objectid,mapc_id,muni_id,muni,parloc_id,poly_typ,map_num,mappar_id,loc_id_cnt,land_value,...,landv_pac,bldgv_psf,totv_pac,bldlnd_rat,sqm_imperv,sqm_bldg,sqm_pave,realesttyp,temp,shape
0,1,1,1,Abington,M_243777_871159,FEE,2,2_5,1.0,100.0,...,3799.689208,-0.999900,3799.689208,0.000000,-0.999900,-0.999900,0.000000,0.0,NaN,3600000004000000080010006A6900001E000000010000...
1,2,2,1,Abington,M_247143_872556,FEE,17,17_23,1.0,196200.0,...,17866.780110,-0.999900,17866.780110,0.000000,-0.999900,-0.999900,0.000000,0.0,NaN,7C00000010000000080010006A690000A4010000010000...
2,3,3,1,Abington,M_247356_872533,FEE,18,18_5,1.0,149700.0,...,176835.192910,76.089109,544090.112587,2.076820,31.810949,13.312941,18.498008,0.0,NaN,7900000010000000080010006A690000A1010000010000...
3,4,4,1,Abington,M_245225_872369,FEE,15,15_103,1.0,164700.0,...,231127.015524,70.778509,593464.571130,1.567699,0.094107,-0.999900,0.000000,0.0,NaN,740000000F000000080010006A6900009C010000010000...
4,5,5,1,Abington,M_245207_872336,FEE,15,15_102,1.0,167500.0,...,237263.406756,52.696986,523962.591995,1.208358,301.967059,182.962820,119.004239,1.0,NaN,690000000D000000080010006A69000091010000010000...


In [5]:
land_parcel_df.iloc [:, [6, 14, 17, 18, 29, 34]]

,map_num,ls_date,luc_1,luc_2,addr_zip,owner_zip
0,2,1.98009e+07,1320,1320,2351,02351
1,17,2.01507e+07,1320,1320,2351,02351
2,18,2.01612e+07,0,0,2351,02351
3,15,2.01411e+07,0,0,2351,02351
4,15,2.01405e+07,1010,1010,2351,02351
...,...,...,...,...,...,...
1274940,28,20130524,1010,1010,01966,01966
1274941,2,19970613,1320,1320,NaN,87506 2667
1274942,9,20150522,1010,1010,NaN,01966
1274943,11,20120619,1010,1010,NaN,01966


In [6]:
land_parcel_df.dtypes

objectid         int64
mapc_id          int64
muni_id          int64
muni            object
parloc_id       object
poly_typ        object
map_num         object
mappar_id       object
loc_id_cnt     float64
land_value     float64
bldg_value     float64
othr_value     float64
total_value    float64
ls_price       float64
ls_date         object
bldg_area      float64
res_area       float64
luc_1           object
luc_2           object
luc_adj_1      float64
luc_adj_2      float64
num_units      float64
units_est      float64
units_src       object
num_rooms      float64
yr_built       float64
site_addr       object
addr_str        object
addr_num        object
addr_zip        object
owner_name      object
owner_addr      object
owner_city      object
owner_stat      object
owner_zip       object
fy             float64
lot_areaft     float64
far            float64
pct_imperv     float64
pct_bldg       float64
pct_pave       float64
landv_pac      float64
bldgv_psf      float64
totv_pac   

In [21]:
# for map_num, addr_zip, and owner_zip default dtype=object is equivalent to string
# map_num has entry 'WAT' that prevents conversion to numeric datatype

# ls_date has non-standard entries, cannot convert to datetime datatype
# land_parcel_df['ls_date'] = pd.to_datetime(land_parcel_df['ls_date'])

# nonstd entries prevents to_numeric conversion, e.g. at position 58545
# land_parcel_df['luc_1'] = pd.to_numeric(land_parcel_df['luc_1'])
# land_parcel_df['luc_2'] = pd.to_numeric(land_parcel_df['luc_2'])

In [19]:
land_parcel_df.loc[58545, ['luc_1']]

luc_1    930V
Name: 58545, dtype: object

In [2]:
def filter_bldg(dataframe):
    '''
    Filter on related columns that indicate whether building(s) are present on the land parcel.
    Removes rows that correspond to land parcels that do not contain buildings.
    Ziba specified: 
        bldg_value - for condos, generally includes land value
        bldg_area - may include garages, stairwells, basements, and other uninhabitable areas.
        bldgv_psf - building value $ per sq foot
    Additional: 
        sqm_bldg - parcel area estimated to be covered by a building (sq meters)
        pct_bldg - % parcel area estimated to be covered by a building 
    '''
    
    return dataframe.query('bldg_value > 0 | \
                           bldg_area > 0 | \
                           bldgv_psf > 0 | \
                           sqm_bldg > 0 | \
                           pct_bldg > 0')

In [28]:
filter_bldg(land_parcel_df)[['bldg_value', 'bldg_area', 'bldgv_psf', 'sqm_bldg', 'pct_bldg']]

,bldg_value,bldg_area,bldgv_psf,sqm_bldg,pct_bldg
2,307400.0,4040.0,76.089109,13.312941,0.388600
3,258200.0,3648.0,70.778509,-0.999900,0.000000
4,199300.0,3782.0,52.696986,182.962820,6.404145
5,206500.0,4752.0,43.455387,129.203406,8.479037
6,276200.0,4634.0,59.602935,236.458540,10.219435
...,...,...,...,...,...
1274938,0.0,0.0,-0.999900,31.267924,2.155103
1274940,296400.0,3136.0,94.515306,12.327159,0.091445
1274942,556400.0,7850.0,70.878981,291.958984,9.852282
1274943,448100.0,5058.0,88.592329,202.132871,21.446002


In [9]:
import pandas as pd 

accepted_codes = ['910','911','912','913','914','915','916','917','918','919','920','921','922', \
                  '923','924','925','926','927','928','929','970','971','972','973','974','975']

#df = pd.read_csv('../data/land_parcel/mapc.ma_parcels_metrofuture.csv')

def filter_luc(dataframe):
    return dataframe[dataframe['luc_1'].isin(accepted_codes) | \
            dataframe['luc_2'].isin(accepted_codes) | \
            dataframe['luc_adj_1'].isin(accepted_codes)| \
            dataframe['luc_adj_2'].isin(accepted_codes)]

#land_parcel_df = filter_luc(df)
land_parcel_df = filter_bldg(land_parcel_df)

In [10]:
land_parcel_df[['luc_1', '']].head(20)

,objectid,mapc_id,muni_id,muni,parloc_id,poly_typ,map_num,mappar_id,loc_id_cnt,land_value,...,landv_pac,bldgv_psf,totv_pac,bldlnd_rat,sqm_imperv,sqm_bldg,sqm_pave,realesttyp,temp,shape
2817,2792,2792,1,Abington,M_246722_873087,FEE,24,24_113,1.0,811600.0,...,2.042666e+05,-0.999900,2.042666e+05,0.000000,13180.877519,124.119733,13056.757786,12.0,NaN,790000000F000000080010006A690000A1010000010000...
4707,4683,4683,1,Abington,M_242973_874629,TAX,36,NaN,1.0,8631400.0,...,1.458055e+04,-0.999900,1.477954e+04,0.013648,24995.422676,179.054199,24816.368477,12.0,NaN,79050000E6000000080118006A690000A1151300010000...
5316,5290,20310,7,Amesbury,M_246978_955401,FEE,77,77-78,1.0,1046000.0,...,1.514202e+05,989.266667,5.810426e+05,2.837285,5763.641017,1821.163525,3942.477492,0.0,NaN,9501000045000000080010006A690000BD051300010000...
5713,5689,20709,7,Amesbury,M_246443_956046,FEE,53,53-195,1.0,2680000.0,...,1.140202e+06,73.047837,3.752456e+06,2.291045,5494.661456,2634.261334,2860.400122,0.0,NaN,E100000021000000080010006A69000089031300010000...
5742,5718,20738,7,Amesbury,M_246444_955991,FEE,65,65-88,1.0,207800.0,...,7.640476e+05,57.747069,2.285157e+06,1.990857,707.788419,514.385795,193.402625,0.0,NaN,3C00000005000000080010006A69000024031300010000...
6068,6039,21059,7,Amesbury,M_246701_958236,FEE,14,14-68,1.0,286000.0,...,2.424546e+05,45.708570,8.432504e+05,2.477972,2594.389782,815.255734,1779.134047,0.0,NaN,3D00000005000000080010006A69000025011300010000...
6555,6527,21547,7,Amesbury,M_246052_956448,FEE,52,52-133,1.0,660000.0,...,8.668161e+05,524.794239,2.206704e+06,1.545758,1671.729789,751.123828,920.605961,0.0,NaN,5400000009000000080010006A6900003C011300010000...
6570,6541,21561,7,Amesbury,M_245992_956328,FEE,52,52-241,1.0,163700.0,...,3.201390e+05,44.315789,8.141348e+05,1.543067,1106.362875,565.303084,541.059791,0.0,NaN,4100000006000000080010006A69000029011300010000...
6571,6542,21562,7,Amesbury,M_246022_956370,FEE,52,52-134,1.0,206500.0,...,9.671723e+04,125.315789,4.312699e+05,3.459080,5754.559035,2066.055028,3688.504006,0.0,NaN,BD0000001B000000080010006A690000A5021300010000...
8686,8653,23673,7,Amesbury,M_246579_956035,FEE,65,65-41,1.0,112400.0,...,4.461601e+05,65.705996,1.803297e+06,3.041815,421.254778,268.178267,153.076511,0.0,NaN,660000000D000000080010006A6900008E011300010000...
